In [ ]:
## IMPORT MODULES

import numpy as np
import random
import math
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import matplotlib.patches as mpatches
import seaborn as sns
np.random.seed(0)

In [ ]:
## SELECT EXPERIMENT TYPE

# p= 0.5
# experiment = 1.1 ## bernoulli p=0.5 [0,1]

# p = 0.8
# experiment = 1.2 # bernoulli p=0.8 [0,1] [0.2,0.8]

# p = 0.2
# experiment = 1.3 # bernoulli p=0.2 [0,1] [0.8,0.2]

# lam = 0.5
# experiment = 2 ## poisson, lam = 0.5 (avg arrivals per unit time)

lam = 0.01
experiment = 2.2 ## poisson, lam = 0.01 (avg arrivals per unit time)

In [ ]:
## EXPERIMENT PARAMETERS

# Define the parameters
N = 20  # Maximum queue length
M = 18   # Maximum multiplier for random value

states_x = list(range(N + 1))
states_y = [0, 1]
actions_u = list(range(2))  # Actions u = {0, 1}
actions_v = [0, 1]  # Actions v = {0, 1}

#arrival_probabilities = [1-p,p]

# Define the probabilities for y

eta = 0.3
prob_y = {
    (0, 0): 1-eta,
    (0, 1): eta,
    (1, 0): eta,
    (1, 1): 1-eta
}

In [ ]:
## TRANSITION PROBABILITIES EXCLUDING CHANNEL STATE

def simulate(x, u, v):
    """Simulate the next state and return the probabilities."""
    transitions = defaultdict(float)
    
    # Calculate all possible transitions for the queue length x
    for k in range(2):
        z = v * k
        x_next = min(N, x - min(x, u) + z)
        transitions[x_next] += arrival_probabilities[k]
    
    return transitions

def calculate_transition_probabilities():
    """Calculate transition probabilities P(s' | s, a) for all (s, a) pairs."""
    transition_probs = defaultdict(lambda: defaultdict(float))

    # Iterate over all possible states and actions
    for x in states_x:
        for u in actions_u:
            for v in actions_v:
                # Simulate the next state probabilities
                transitions = simulate(x,u,v)
                for next_x, prob in transitions.items():
                    transition_probs[(x,u,v)][next_x] += round(prob,2)

    return transition_probs

# Calculate the transition probabilities for all state-action pairs
transition_probs = calculate_transition_probabilities()


def plot_transition_probabilities(transition_probs, states_x, actions_u, actions_v):
    """Plot the transition probabilities."""
    fig, axes = plt.subplots(len(actions_u), len(actions_v), figsize=(15, 10), sharex=True, sharey=True)
    if len(actions_u) == 1 and len(actions_v) == 1:
        axes = [[axes]]
    elif len(actions_u) == 1 or len(actions_v) == 1:
        axes = [axes]

    for i, u in enumerate(actions_u):
        for j, v in enumerate(actions_v):
            # Create a matrix of transition probabilities
            prob_matrix = np.zeros((len(states_x), len(states_x)))
            for x in states_x:
                for next_x, prob in transition_probs[(x, u, v)].items():
                    prob_matrix[x, next_x] = prob

            ax = axes[i][j]
            sns.heatmap(prob_matrix, annot=False, fmt=".2f", cmap="viridis", ax=ax)
                     
            # Annotate only the center and corner squares
            for x in [0, len(states_x) - 1, len(states_x) // 2]:
                for y in [0, len(states_x) - 1, len(states_x) // 2]:
                    if x < len(states_x) and y < len(states_x):  # Ensure indexes are within bounds
                        ax.text(y+0.5, x+0.5, f"{prob_matrix[x, y]:.1f}", 
                                ha='center', va='center', color='black')
           
            ax.set_title(f"u={u}, v={v}")
            ax.set_xlabel("Next State")
            ax.set_ylabel("Current State")
    plt.tight_layout()
    plt.savefig(f"Experiment{experiment}_transitions.png",dpi=300)
    plt.show()

# Plot the transition probabilities
plot_transition_probabilities(transition_probs, states_x, actions_u, actions_v)

In [ ]:
## TRANSITION PROBABILITIES INCLUDING CHANNEL STATE


def simulate(x, y, u, v):
    """Simulate the next state and return the probabilities."""
    transitions = defaultdict(float)
    
    # Calculate all possible transitions for the queue length x
    for a, p_a in zip([0, 1], arrival_probabilities):
        z = a * v
        x_next = min(N, x - min(x, u) + z)
        transitions[(x_next, 0)] += prob_y[(y, 0)] * p_a
        transitions[(x_next, 1)] += prob_y[(y, 1)] * p_a
    
    return transitions

def calculate_transition_probabilities():
    """Calculate transition probabilities P(s' | s, a) for all (s, a) pairs."""
    transition_probs = defaultdict(lambda: defaultdict(float))

    # Iterate over all possible states and actions
    for x in states_x:
        for y in states_y:
            for u in actions_u:
                for v in actions_v:
                    # Simulate the next state probabilities
                    transitions = simulate(x, y, u, v)
                    for (next_x, next_y), prob in transitions.items():
                        transition_probs[(x, y, u, v)][(next_x, next_y)] += prob
    
    return transition_probs


# Calculate the transition probabilities
transition_probs = calculate_transition_probabilities()

# Print the transition probabilities for verification
for state_action, transitions in transition_probs.items():
    print(f"From state-action {state_action}:")
    for next_state, prob in transitions.items():
        print(f"  To next state {next_state} with probability {prob:.4f}")

# Convert transition_probs dictionary to a list of tuples
transition_probs_list = []

for state_action, transitions in transition_probs.items():
    for next_state, prob in transitions.items():
        transition_probs_list.append((*state_action, *next_state, prob))


# Function to plot heatmaps
def plot_heatmap(probabilities, title):
    plt.figure(figsize=(8, 6))
    plt.imshow(probabilities, cmap='viridis', interpolation='nearest', vmin=0, vmax=1)
    plt.colorbar()
    plt.xlabel('Next State (x)')
    plt.ylabel('Current State (x)')
    plt.title(title)
    plt.xticks(np.arange(len(states_x)), states_x)
    plt.yticks(np.arange(len(states_x)), states_x)
    
    plt.gca().invert_yaxis()  # Invert y-axis to match matrix representation
    # Add annotations for probability values in specific positions
    for i, j in [(0, 0), (0, len(states_x)//2), (0, len(states_x)-1),
                 (len(states_x)//2, 0), (len(states_x)//2, len(states_x)//2),
                 (len(states_x)//2, len(states_x)-1),
                 (len(states_x)-1, 0), (len(states_x)-1, len(states_x)//2),
                 (len(states_x)-1, len(states_x)-1)]:
        text = f'{probabilities[i, j]:.2f}'  # Format probability to 2 decimal places
        plt.text(j, i, text, ha='center', va='center', color='white')


    plt.tight_layout()
    plt.show()


# Iterate over each (y, u, v) tuple and plot heatmap

for nexty in [0]:
    for y in [0]:
        for u in actions_u:
            for v in actions_v:
                # Initialize a matrix to store probabilities
                probabilities_matrix = np.zeros((len(states_x), len(states_x)))

                # Find the relevant entries in transition_probs_list for (y, u, v)
                relevant_entries = [(x, next_x, prob) for x, y_, u_, v_, next_x, nexty_, prob 
                                    in transition_probs_list if y == y_ and u == u_ and v == v_ and nexty == nexty_]

                # Populate the matrix with transition probabilities
                for x, next_x, prob in relevant_entries:
                    probabilities_matrix[x, next_x] = prob

                # Plot the heatmap
                title = f"Heatmap for (y={y}, y_next = {nexty}, u={u}, v={v})"
                plot_heatmap(probabilities_matrix, title)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming `states_x`, `actions_u`, `actions_v`, and `transition_probs_list` are already defined.

# Function to plot heatmaps
def plot_heatmap(ax, probabilities, subtitle):
    cax = ax.imshow(probabilities, cmap='viridis', interpolation='nearest', vmin=0, vmax=1)
    ax.set_xlabel('Next State (x)')
    ax.set_ylabel('Current State (x)')
    ax.set_title(subtitle, fontsize=10)
    ax.set_xticks(np.arange(len(states_x)))
    ax.set_yticks(np.arange(len(states_x)))
    ax.set_xticklabels(states_x)
    ax.set_yticklabels(states_x)

    # Add annotations for probability values in specific positions
    for i, j in [(0, 0), (0, len(states_x)//2), (0, len(states_x)-1),
                 (len(states_x)//2, 0), (len(states_x)//2, len(states_x)//2),
                 (len(states_x)//2, len(states_x)-1),
                 (len(states_x)-1, 0), (len(states_x)-1, len(states_x)//2),
                 (len(states_x)-1, len(states_x)-1)]:
        text = f'{probabilities[i, j]:.2f}'  # Format probability to 2 decimal places
        ax.text(j, i, text, ha='center', va='center', color='white')
    return cax

# Number of (y, nexty) pairs
num_rows = 4

# Number of (u, v) pairs
num_cols = len(actions_u) * len(actions_v)

# Create a larger grid of subplots
fig, axs = plt.subplots(num_rows, num_cols, figsize=(num_cols * 4, num_rows * 4))
axs = axs.reshape(num_rows, num_cols)  # Reshape axs to a 2D array for easy indexing

# Initialize a counter for subplot index
subplot_idx = 0

# Iterate over each (nexty, y) pair and plot heatmaps
row_idx = 0
for nexty in [0, 1]:
    for y in [0, 1]:
        col_idx = 0
        for u in actions_u:
            for v in actions_v:
                # Initialize a matrix to store probabilities
                probabilities_matrix = np.zeros((len(states_x), len(states_x)))

                # Find the relevant entries in transition_probs_list for (y, u, v)
                relevant_entries = [(x, next_x, prob) for x, y_, u_, v_, next_x, nexty_, prob 
                                    in transition_probs_list if y == y_ and u == u_ and v == v_ and nexty == nexty_]

                # Populate the matrix with transition probabilities
                for x, next_x, prob in relevant_entries:
                    probabilities_matrix[x, next_x] = prob

                # Plot the heatmap on the current subplot
                subtitle = f"u={u}, v={v}"
                cax = plot_heatmap(axs[row_idx, col_idx], probabilities_matrix, subtitle)
                col_idx += 1

        # Add a title above the current row of subplots
        fig.text(0.5, 1 - (row_idx+1)*(1)/ num_rows + 0.02, f'Heatmaps for y={y}, y_next={nexty}', ha='center', va='bottom', fontsize=16, transform=fig.transFigure, color='orange')
        row_idx += 1

# Adjust layout to make space for the titles and colorbar
fig.tight_layout(rect=[0, 0, 1, 0.95])

# Add a colorbar to the side
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])  # Position: [left, bottom, width, height]
fig.colorbar(cax, cax=cbar_ax)
plt.savefig(f'Experiment{experiment}_transitions_withY.png',dpi=300)
plt.show()





In [ ]:
### COST FUNCTIONS

def cost(q):
    return np.exp((np.maximum(q-14, 0)))

# Generate q values over the entire range
marker_q_values = np.arange(N+1)
q_values = np.array(marker_q_values)

# Plot the reward function with markers
plt.figure(figsize=(10, 6))

# Plot the regular function
plt.plot(q_values, cost(q_values), label='Cost Function')

# Add markers at specific values of q

plt.scatter(marker_q_values, cost(np.array(marker_q_values)), color='red')

# Labeling and grid
plt.title(f'Cost Function for N={N} and M={M}')
plt.xlabel('q')
plt.xticks(np.arange(N+1))
plt.ylabel(r'Cost = $e^{(q-14)^{+}}$', fontsize=12) 
plt.grid(True)

# Show legend
plt.legend()
plt.savefig(f'Experiment{experiment}_cost.png',dpi=300)
plt.show()



eta = 0.3
# Define Markov chain transition probabilities for the slower Q-learning
transition_matrix = {0: {0: 1 - eta, 1: eta}, 1: {0: eta, 1: 1 - eta}}

# Function to update Markov chain state for the slower Q-learning
def update_markov_chain_state(current_state):
    next_state = np.random.choice(list(transition_matrix[current_state].keys()), 
                                p=list(transition_matrix[current_state].values()))
    return next_state


########### COST FUNCTION for Slow Q Learning:

# Function to calculate the cost for the slower Q-learning
def k(i, u, y):
    if y == 0:
        a = 1
    elif y == 1:
        a = 6
    # Adjusted value for C

    term1 = ((a * min(i, u)) ** 2) 
    term2 = 2*(i)
    return term1+term2

i_values = np.arange(0, N+1)
u_values = np.arange(2)

# Calculate costs to determine the color scale limits
costs_all = []
for y in [0, 1]:
    costs = np.zeros((N+1, 2))
    for i in i_values:
        for u in u_values:
            costs[i, u] = k(i, u, y)
    costs_all.append(costs)

# Determine the global min and max costs for color scaling
global_min = np.min(costs_all)
global_max = np.max(costs_all)

fig, axs = plt.subplots(1, 2, figsize=(16, 8))

for y in [0, 1]:
    costs = costs_all[y]
    state_label = 'good' if y == 0 else 'bad'
    
    ax = axs[y]
    c = ax.pcolormesh(u_values, i_values, costs, shading='auto', cmap='viridis', vmin=global_min, vmax=global_max)
    ax.set_title(f'Cost function for y={y} ({state_label} state)', fontsize=20)
    ax.set_xlabel('Departure Action u', fontsize=20)
    ax.set_ylabel('Queue length i', fontsize=20)
    ax.set_xticks(u_values)
    ax.set_yticks(i_values)
    ax.tick_params(axis='both', which='major', labelsize=14)
    fig.colorbar(c, ax=ax)

# Set the same y-axis limits and y-ticks for both plots
for ax in axs:
    ax.set_xlim(0, 1)
    ax.set_ylim(0, N)
    ax.set_yticks(np.arange(0, N+1, 5))  # Set y-ticks at intervals of 5 for clarity

plt.tight_layout()
plt.savefig(f'RiskClassic_classicalcost.png',dpi=300)
plt.show()

In [ ]:
### MIN MIN FUNCTION


def minmin(Q, X):
    # Extract the slice of Q for the given X
    Q_slice = Q[X, :, :]
    
    # Find the minimum value
    min_value = np.min(Q_slice)
    
    # Find indices of the minimum value
    min_indices = np.where(Q_slice == min_value)
    
    # Prepare a list of indices
    min_indices_list = list(zip(min_indices[0], min_indices[1]))
    
    return min_value, min_indices_list

def minmin2(Q, X,Y):
    # Extract the slice of Q for the given X
    Q_slice = Q[X,Y, :, :]
    
    # Find the minimum value
    min_value = np.min(Q_slice)
    
    # Find indices of the minimum value
    min_indices = np.where(Q_slice == min_value)
    
    # Prepare a list of indices
    min_indices_list = list(zip(min_indices[0], min_indices[1]))
    
    return min_value, min_indices_list

In [ ]:
######## Q VALUE ITERATION ############

delta_store = []
time_store = []

def q_value_iteration(states_x, actions_v, transition_probs, gamma=1, theta=1e-6):
    """Perform Q-value iteration to find the optimal action-value function."""
    # Initialize the Q-values arbitrarily

    #Q = np.multiply(100,np.random.rand(N + 1,2, M + 1,2))
    Q = np.multiply(np.random.rand(N + 1,2,2),100)

    x = 0
    u = 0
    v = 0
    
    # Iterate until convergence
    
    step=0
    while True:
        delta = 0

        for x in states_x:
            #for y in states_y:
               # for u in actions_u:
            for u in actions_u:
                for v in actions_v:

                    old_q_value = Q[x,u,v]
                    new_q_value = 0

                    # Calculate the new Q-value using the Bellman equation

                    normalization_factor = 1
                    sum_prob_S0 = sum(transition_probs[x,u,v][j] for j in np.arange(M+1) if j in transition_probs[x,u,v])
                    cost_term = cost(x)

                    for next_x, prob in transition_probs[(x, u,v)].items():

                        #miniminQ, _, _ = minmin(Q,next_x,next_y)
                        miniminQ = np.min(Q[next_x,:])
                        #new_q_value += prob *  ((identity * cost(x) * miniminQ)/Q[0,0]) 
                        new_q_value += prob * miniminQ

                    # if x==18 and u==0:
                    #     #print("step", step, "v",v, "min_sum", new_q_value, "s0_sum", sum_prob_S0)
                    #     print("___")
                    new_q_value = new_q_value * cost_term * sum_prob_S0/normalization_factor

                
                    
                    # Update the Q-value
            
                    Q[x,u,v] = new_q_value
                
                    #print(new_q_value-q)
                    #print(x,y,u,v,Q[x,y,u,v])
                    delta = max(delta,old_q_value-new_q_value)
                    
        
                     
        
        #if step%10==0:
        print("step", step, "del",delta)
        delta_store.append(delta)
        time_store.append(step)

    
        step+=1
        # Check for convergence
        
        if delta < theta:
            break


    return Q

optimal_q_values = q_value_iteration(states_x, actions_v, transition_probs)

In [ ]:
###### VALUE ITERATION CONVERGENCE

# Create subplots
fig, axs = plt.subplots(1, 2, figsize=(8, 4))

# Plot the first dataset
axs[0].plot(time_store, delta_store)
#axs[0].set_title('Delta (Max Change in Q-Value per Iteration) over Time')
axs[0].set_xlabel('Time Steps', fontsize=14)
axs[0].set_ylabel('Delta', fontsize=14)
axs[0].set_xticks(range(0, len(delta_store) + 1, 100))
axs[0].grid(False)

# Plot the second dataset
axs[1].plot(time_store[10:], delta_store[10:])
axs[1].set_xlabel('Time Steps', fontsize=14)
axs[1].set_ylabel('Delta', fontsize=14)
axs[1].set_xticks(range(10, len(delta_store) + 1, 100))
axs[1].grid(False)

# Adjust layout to prevent overlapping titles
plt.tight_layout()

# Save the figure
plt.savefig(f'Experiment{experiment}_delta_subplots.png', dpi=300)

# Show the plots
plt.show()

In [ ]:
#### LEARNING RATES #########

# Define learning rates as functions of 'a(n)' and 'b(n)'
c1 = 0.001

def a_learning_rate(n):
    return c1 / (1 + math.ceil(n /125))

# Define learning rates as functions of 'a(n)' and 'b(n)'
def b_learning_rate(n):
    return c1 / (1 + math.ceil(((n/25+1)*np.log(n/25+1)) /250))

In [ ]:
#### GENERATING POISSON ARRIVALS

def generate_poisson_arrivals(lam, duration_steps):
    arrivals = np.zeros(duration_steps, dtype=int)
    time = 0
    while time < duration_steps:
        interval = np.random.exponential(scale=1/lam)
        time += interval
        if int(time) < duration_steps:
            arrivals[int(time)] = 1
    return arrivals

 # Rate of lam events per time step
duration_steps = 1000000 # Total duration of 100 time steps

arrivals = generate_poisson_arrivals(lam, duration_steps)
print(arrivals[1000:1100])


In [ ]:

# Aggregate arrivals into bins
bin_size = 10000//2
# Aggregate arrivals into bins
num_bins = duration_steps // bin_size
binned_arrivals = np.add.reduceat(arrivals, np.arange(0, duration_steps, bin_size))

# Plotting
plt.figure(figsize=(12, 6))
bars = plt.bar(np.arange(num_bins) * bin_size, binned_arrivals, width=bin_size, color='b', alpha=0.7)

# Annotate plot
plt.xlabel('Time Step')
plt.ylabel('Number of Arrivals')
plt.title(f'Poisson Arrival Distribution (Rate λ={lam})')
plt.grid(True)
plt.tight_layout()

# Annotate the bin size
plt.text(0.02, 0.95, 'Bin size: {}'.format(bin_size), transform=plt.gca().transAxes,
         fontsize=12, verticalalignment='top', bbox=dict(boxstyle="round,pad=0.3", edgecolor='black', facecolor='white'))

# Annotate the max number of arrivals in any bin
max_arrivals = max(binned_arrivals)
max_bin = np.argmax(binned_arrivals) * bin_size
plt.annotate('Max Arrivals: {}'.format(max_arrivals),
             xy=(max_bin, max_arrivals), xytext=(max_bin + duration_steps * 0.05, max_arrivals * 1.1),
             arrowprops=dict(facecolor='red', shrink=0.05),
             fontsize=12, color='red')

plt.savefig(f'Experiment{experiment}_arrivalsim.png',dpi=300)
plt.show()

In [ ]:
from scipy.stats import expon
def generate_inter_arrival_times(lam, duration_steps):
    inter_arrival_times = []
    time = 0
    while time < duration_steps:
        interval = np.random.exponential(scale=1/lam)
        inter_arrival_times.append(interval)
        time += interval
    return inter_arrival_times

# Parameters
lam = 0.5  # Poisson rate
duration_steps = 1000000  # Total time steps

# Generate inter-arrival times
inter_arrival_times = generate_inter_arrival_times(lam, duration_steps)

# Theoretical exponential distribution
x_values = np.linspace(0, max(inter_arrival_times), 1000)
pdf_values = expon.pdf(x_values, scale=1/lam)

# Plotting
plt.figure(figsize=(12, 6))
plt.hist(inter_arrival_times, bins=100, density=True, alpha=0.7, color='b', label='Simulated Inter-Arrival Times')
plt.plot(x_values, pdf_values, 'r-', label='Theoretical Exponential Distribution')
plt.xlabel('Time Between Arrivals')
plt.ylabel('Probability Density')
plt.title('Inter-Arrival Time Distribution (Rate λ={})'.format(lam))
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig(f'Experiment{experiment}_arrivaltime.png',dpi=300)
plt.show()

In [ ]:
####### Q LEARNING ##############

delta_store = []
time_store = []
changer = [[[[None for _ in range(0)] for _ in range(2)] for _ in range(2)] for _ in range(N + 1)]
def q_learning(states_x, actions_v, transition_probs, gamma=1, theta=1e-5):
    """Perform Q-value iteration to find the optimal action-value function."""
    # Initialize the Q-values arbitrarily

    Q_fast = np.multiply(np.random.rand(N + 1,2,2)+1,100)

    # Iterate until convergence
    
    step=0
    while True:
        delta = 0

        alpha_slow = a_learning_rate(step)

        for x in states_x:
            #for y in states_y:
               # for u in actions_u:
            for u in actions_u:
                for v in actions_v:



                    old_q_value = Q_fast[x,u,v]
                    new_q_value = 0
                    arrival = arrivals[step]
                    x_next = min(N, x - min(u,x) + v*arrival)
                    normalization_factor = 1
                    cost_term = cost(x)
                    if x_next < M+1:
                        identity = 1
                        miniminQ, _ = minmin(Q_fast, x_next)

                        change = alpha_slow * (identity * cost_term * miniminQ)/normalization_factor
                    else:
                        identity=0
                        change = 0
                    new_q_value = (1-alpha_slow)*old_q_value + alpha_slow*(change)

                    changer[x][u][v].append(abs(alpha_slow*(change - old_q_value)))
            
                    Q_fast[x,u,v] = new_q_value


              



                    delta = max(delta,old_q_value-new_q_value)
                    
        
                     
        
        #if step%10==0:
        print("step", step, "del",delta)
        delta_store.append(delta)
        time_store.append(step)

    
        step+=1
        # Check for convergence
        
        if delta < theta:
            break


    return Q_fast

optimal_q_learning  = q_learning(states_x, actions_v, transition_probs)
print(optimal_q_learning)

In [ ]:
experiment

In [ ]:
for x in range(21):
    for u in range(2):
        for v in range(2):
            print(f"x={x}, u={u}, v={v}, q-value, {optimal_q_learning[x,u,v]}")
    print(f"u,v = {minmin(optimal_q_learning,x)[-1]}")
    print("____")

In [ ]:
###### Q LEARNING CONVERGENCE

# Plotting
for x in range(M + 1):
    fig, axs = plt.subplots(1,2, figsize=(14, 5))  # Create a figure with 2 subplots side by side

    # Plot with changer[x][u][v][10000:]
    for u in range(2):
        for v in range(2):
            axs[0].plot(range(len(changer[x][u][v][:])), changer[x][u][v][:], label=f'u={u}, v={v}')
    axs[0].set_title(f'Plots for x={x} (Entire Data)')
    axs[0].set_xlabel('Index')
    axs[0].set_ylabel('Value')
    axs[0].legend()

    # Plot with changer[x][u][v][:]
    for u in range(2):
        for v in range(2):
            axs[1].plot(range(len(changer[x][u][v][10000:])), changer[x][u][v][10000:], label=f'u={u}, v={v}')
    axs[1].set_title(f'Plots for x={x} (10000:)')
    axs[1].set_xlabel('Index')
    axs[1].set_ylabel('Value')
    axs[1].legend()

    plt.savefig(f"Experiment{experiment}_qlearning_subplots{x}.png", dpi=300)

    plt.show()

In [ ]:
########## DUAL Q LEARNING 


############ RISK + CLASSICAL Q LEARNING #################

delta_store = []
delta_store_slow = []
time_store = []
changer = [[[[[None for _ in range(0)] for _ in range(2)] for _ in range(2)] for _ in range(2)] for _ in range(N + 1) ]


changer_slow = [[[[[None for _ in range(0)] for _ in range(2)] for _ in range(2)] for _ in range(2)] for _ in range(N + 1) ]

def dual_q_learning(beta=0.9, theta=1e-5):
    """Perform Q-value iteration to find the optimal action-value function."""
    # Initialize the Q-values arbitrarily

    Q_fast = np.multiply(np.random.rand(N + 1,2,2,2)+1,100)
    Q_slow = np.multiply(np.random.rand(N+1,2,2)+1, 100)

    # Iterate until convergence
    
    step=0
    while True:
        delta = 0
        delta_slow = 0

        a_n = a_learning_rate(step)
        b_n = b_learning_rate(step)

        for x in states_x:
            for y in states_y:
                for u in actions_u:
                    for v in actions_v:



######################################################################################
                        old_q_value = Q_fast[x,y,u,v]
                        new_q_value = 0
                        arrival = np.random.choice([0, 1], p=[1-p, p])
                        x_next = min(N, x - min(u,x) + v*arrival)
                        y_next = update_markov_chain_state(y)
                        normalization_factor = 1
                        cost_term = cost(x)
                        if x_next < M+1:
                            identity = 1
                            miniminQ, _ = minmin2(Q_fast, x_next,y_next)

                            change = a_n * (identity * cost_term * miniminQ)/normalization_factor
                        else:
                            identity=0
                            change = 0
                        new_q_value = (1-a_n)*old_q_value + a_n*(change)

                        changer[x][y][u][v].append(abs(a_n*(change - old_q_value)))
                
                        Q_fast[x,y,u,v] = new_q_value


                        delta = max(delta,old_q_value-new_q_value)
######################################################################################

######################################################################################

                        if step%25==0:

                            old_q_value_slow = Q_slow[x,y,u]
                            new_q_value_slow = 0
                            classic_cost_term = k(i,y,u)

                            if identity==1:

                                norm_factor = minmin2(Q_fast,0,0)[0]
                                numerator = identity * minmin2(Q_fast, x_next, y_next)[0]
                                denominator = minmin2(Q_fast,x,y)[0]*norm_factor
                                big_term = numerator/denominator

                                change_slow = classic_cost_term + beta*big_term*np.min(Q_slow[x_next,y_next,:])
                            else:
                                change_slow = classic_cost_term 

                            new_q_value_slow = (1-b_n)*(old_q_value_slow) + b_n*(change_slow)


                            changer_slow[x][y][u][v].append(abs(b_n*(change_slow - old_q_value_slow)))

                            Q_slow[x,y,u] = new_q_value_slow

                            delta_slow = max(delta_slow, old_q_value_slow - new_q_value_slow)
######################################################################################

                        
        if step%25==0:
            print("step", step, "del",delta, "del slow", delta_slow)
            delta_store_slow.append(delta_slow)
        delta_store.append(delta)
        time_store.append(step)

    
        step+=1
        # Check for convergence
        
        if delta < theta and delta_slow < theta*10:
            break


    return Q_fast, Q_slow

optimal_dual_q_learning  = dual_q_learning()
print(optimal_dual_q_learning)

In [ ]:
######### DUAL Q LEARNING CONVERGENCE


# Plotting
for x in range(M + 1):
    fig, axs = plt.subplots(2, 2, figsize=(14, 10))  # Create a figure with 2x2 subplots

    # Plot changer[x][u][v][:]
    for u in range(2):
        for v in range(2):
            axs[0, 0].plot(range(len(changer[x][y][u][v][:])), changer[x][y][u][v][:], label=f'u={u}, v={v}')
    axs[0, 0].set_title(f'Plots for changer, x={x} (Entire Data)')
    axs[0, 0].set_xlabel('Index')
    axs[0, 0].set_ylabel('Value')
    axs[0, 0].legend()

    # Plot changer[x][u][v][10000:]
    for u in range(2):
        for v in range(2):
            axs[0, 1].plot(range(10000, len(changer[x][y][u][v][:])), changer[x][y][u][v][10000:], label=f'u={u}, v={v}')
    axs[0, 1].set_title(f'Plots for changer, x={x} (10000:)')
    axs[0, 1].set_xlabel('Index')
    axs[0, 1].set_ylabel('Value')
    axs[0, 1].legend()

    # Plot changer_slow[x][u][v][:]
    for u in range(2):
        for v in range(2):
            axs[1, 0].plot(range(len(changer_slow[x][y][u][v][:])), changer_slow[x][y][u][v][:], label=f'u={u}, v={v}')
    axs[1, 0].set_title(f'Plots for changer_slow, x={x} (Entire Data)')
    axs[1, 0].set_xlabel('Index')
    axs[1, 0].set_ylabel('Value')
    axs[1, 0].legend()

    # Plot changer_slow[x][u][v][10000:]
    for u in range(2):
        for v in range(2):
            axs[1, 1].plot(range(10000, len(changer_slow[x][y][u][v][:])), changer_slow[x][y][u][v][10000:], label=f'u={u}, v={v}')
    axs[1, 1].set_title(f'Plots for changer_slow, x={x} (10000:)')
    axs[1, 1].set_xlabel('Index')
    axs[1, 1].set_ylabel('Value')
    axs[1, 1].legend()

    plt.tight_layout()  # Adjust layout automatically to prevent overlap

    plt.savefig(f"Experiment{experiment}_dual_qlearning_subplots{x}.png", dpi=300)
    plt.show()


In [ ]:
####### DUAL Q LEARNING POLICY 

Q_fast, Q_slow = optimal_dual_q_learning


def find_optimal_pairs(Q_slow, Q_fast):
   
    optimal_pairs = []
    
    for x in range(N+1):
        for y in range(2):
            # Find all indices v that minimize Q_fast(x, y, u, v)
            v_indices = np.where(Q_fast[x, y].sum(axis=0) == np.min(Q_fast[x, y].sum(axis=0)))[0]
            
            for v_index in v_indices:
                # Find all indices u that minimize Q_slow(x, y, u)
                u_indices = np.where(Q_slow[x, y] == np.min(Q_slow[x, y]))[0]
                
                for u_index in u_indices:
                    # Store optimal pair (u_index, v_index)
                    optimal_pairs.append((x, y, u_index, v_index))
    
    return optimal_pairs

# Example function call
optimal_pairs = find_optimal_pairs(Q_slow, Q_fast)

def print_optimal_pairs(optimal_pairs):
    # Iterate over unique (x, y) pairs
    x_values = set(pair[0] for pair in optimal_pairs)
    y_values = set(pair[1] for pair in optimal_pairs)
    
    for x in x_values:
        for y in y_values:
            # Filter pairs for current (x, y) pair
            pairs_for_xy = [(u, v) for (x_, y_, u, v) in optimal_pairs if x_ == x and y_ == y]
            
            # Print results for current (x, y) pair
            print(f"Optimal pairs for (x={x}, y={y}):")
            for (u, v) in pairs_for_xy:
                print(f"  u = {u}, v = {v}")
                print("___")


print_optimal_pairs(optimal_pairs)

In [ ]:
########### VALUE ITERATION AND Q LEARNING COMPARISON 



# Normalize Q-values for each queue length separately between 0 and 1
def normalize_q_values(optimal_q_values):
    optimal_q_values_normalized = np.zeros_like(optimal_q_values)
    for x in range(21):
        q_values = optimal_q_values[x, :, :].flatten()  # Flatten Q-values for all pairs (u,v) for this queue length
        q_min, q_max = q_values.min(), q_values.max()
        if q_min != q_max:  # Avoid division by zero
            optimal_q_values_normalized[x, :, :] = (optimal_q_values[x, :, :] - q_min) / (q_max - q_min)
        else:
            optimal_q_values_normalized[x, :, :].fill(0.5)  # Set all normalized Q-values to 0.5 if they are all equal
    return optimal_q_values_normalized

# Normalized Q-values
optimal_q_values_normalized = normalize_q_values(optimal_q_values)
optimal_q_learning_normalized = normalize_q_values(optimal_q_learning)

# Create figure with two subplots
fig, axs = plt.subplots(1, 2, figsize=(20, 6))

# Plot each pair in the first subplot
colors = ['#FF4D4D', '#006400', 'red', '#008080']
labels = ['(u=0, v=0)', '(u=0, v=1)', '(u=1, v=0)', '(u=1, v=1)']

for i, (u, v) in enumerate([(0, 0), (0, 1), (1, 0), (1, 1)]):
    axs[0].scatter(range(19), optimal_q_values_normalized[:19, u, v], color=colors[i], label=labels[i], linewidth=2, marker="o", s=100)

axs[0].set_title('Q-values (Normalized) for Different Pairs (Q-Value Iteration)', fontsize=16)
axs[0].set_xlabel('Queue length', fontsize=14)
axs[0].set_ylabel('Normalized Q-value', fontsize=14)
axs[0].set_xticks(range(19))
axs[0].legend(loc='upper center')
axs[0].grid(False)

# Plot each pair in the second subplot
for i, (u, v) in enumerate([(0, 0), (0, 1), (1, 0), (1, 1)]):
    axs[1].scatter(range(19), optimal_q_learning_normalized[:19, u, v], color=colors[i], label=labels[i], linewidth=2, marker="x", s=100)

axs[1].set_title('Q-values (Normalized) for Different Pairs (Q-Learning)', fontsize=16)
axs[1].set_xlabel('Queue length', fontsize=14)
axs[1].set_ylabel('Normalized Q-value', fontsize=14)
axs[1].set_xticks(range(19))
axs[1].legend(loc='upper center')
axs[1].grid(False)

# Save the combined figure
plt.savefig(f'Combined_Experiment{experiment}.png', dpi=300)
plt.show()

In [ ]:
######## DUAL Q LEARNING - QUEUE TRAJECTORY


y_init = 0
for i_init in [0,4,6,8,10,14,15,16,17,18]:

    i_traj = []
    actions_traj = []

    i = i_init
    y = y_init
    time=0
    for time in range(60):

        (u,v) = [(u, v) for (x_, y_, u, v) in optimal_pairs if x_ == i and y_ == y][0]

        i_traj.append(i)
        actions_traj.append((u,v))
        print(i)
        print((u,v))
        i = min(N, i - min(i,u) + v*np.random.choice(2))
        y = update_markov_chain_state(y)


    # Number of time steps
    time_steps = range(len(i_traj))

    # Create figure and axes
    fig, ax = plt.subplots(figsize=(14, 8))

    # Plot the queue length over time
    ax.plot(time_steps, i_traj, marker='o', linestyle='-', color='b', label='Queue Length')

    # Initialize legend handles and labels for the arrows
    legend_handles = []
    legend_labels = []

    # Annotate the actions on the plot
    for t, (i, action) in enumerate(zip(time_steps, actions_traj)):
        u, v = action
        
        # Annotate departures when u = 1
        if u == 1:
            if v == 0:
                # Handle simultaneous departure and arrival blocking with different arrow styles
                arrow_style = '-|>'
                arrow_color = 'red'
                label = 'Departure & Blocked Arrival (u=1, v=0)'
            else:
                arrow_style = '->'
                arrow_color = 'green'
                label = 'Departure (u=1)'
        # Annotate arrival blocking when v = 0
        elif v == 0:
            arrow_style = '-|>'
            arrow_color = 'purple'
            label = 'Arrival Blocking (v=0)'
        else:
            continue  # Skip if neither u nor v is 0
        
        # Annotate arrows
        ax.annotate("", xy=(t, i_traj[t]), xytext=(0, 20), textcoords='offset points', arrowprops=dict(arrowstyle=arrow_style, linewidth=2, color=arrow_color))
        
        # Append legend handles and labels for the arrows
        legend_handles.append(mpatches.Patch(color=arrow_color))
        legend_labels.append(label)

    # Add legend for the arrows
    ax.legend(handles=legend_handles, labels=legend_labels, loc='upper left', fontsize=10)

    # Create color legend for the arrow explanations
    color_legend_handles = [mpatches.Patch(color='red', label='Departure & Blocked Arrival (u=1, v=0)'),
                            mpatches.Patch(color='green', label='Departure (u=1)'),
                            mpatches.Patch(color='purple', label='Arrival Blocking (v=0)')]
    ax.legend(handles=color_legend_handles, loc='lower right', fontsize=10)

    # Set labels and title
    ax.set_xlabel('Time', fontsize=12)
    ax.set_ylabel('Queue Length', fontsize=12)
    ax.set_title(f'Queue Length Over Time, Initial length i = {i_init}', fontsize=14)
    ax.grid(True)

    # Show plot
    plt.savefig(f'Experiment{experiment}_queue{i_init}_RISKANDCLASSIClearning.png',dpi=300)
    plt.show()

In [ ]:
########### Q LEARNING / VALUE ITERATION - QUEUE TRAJECTORY 



for i_init in [0,4,6,8,10,14,15,16,17,18]:

    i_traj = []
    actions_traj = []

    i = i_init
    time=0
    for time in range(60):

        (u,v) = minmin(optimal_q_learning, i)[-1][np.random.choice(len(minmin(optimal_q_learning, i)[-1]))]

        i_traj.append(i)
        actions_traj.append((u,v))
        print(i)
        print((u,v))
        i = min(N, i - min(i,u) + v*np.random.choice(2))


    # Number of time steps
    time_steps = range(len(i_traj))

    # Create figure and axes
    fig, ax = plt.subplots(figsize=(14, 8))

    # Plot the queue length over time
    ax.plot(time_steps, i_traj, marker='o', linestyle='-', color='b', label='Queue Length')

    # Initialize legend handles and labels for the arrows
    legend_handles = []
    legend_labels = []

    # Annotate the actions on the plot
    for t, (i, action) in enumerate(zip(time_steps, actions_traj)):
        u, v = action
        
        # Annotate departures when u = 1
        if u == 1:
            if v == 0:
                # Handle simultaneous departure and arrival blocking with different arrow styles
                arrow_style = '-|>'
                arrow_color = 'red'
                label = 'Departure & Blocked Arrival (u=1, v=0)'
            else:
                arrow_style = '->'
                arrow_color = 'green'
                label = 'Departure (u=1)'
        # Annotate arrival blocking when v = 0
        elif v == 0:
            arrow_style = '-|>'
            arrow_color = 'purple'
            label = 'Arrival Blocking (v=0)'
        else:
            continue  # Skip if neither u nor v is 0
        
        # Annotate arrows
        ax.annotate("", xy=(t, i_traj[t]), xytext=(0, 20), textcoords='offset points', arrowprops=dict(arrowstyle=arrow_style, linewidth=2, color=arrow_color))
        
        # Append legend handles and labels for the arrows
        legend_handles.append(mpatches.Patch(color=arrow_color))
        legend_labels.append(label)

    # Add legend for the arrows
    ax.legend(handles=legend_handles, labels=legend_labels, loc='upper left', fontsize=10)

    # Create color legend for the arrow explanations
    color_legend_handles = [mpatches.Patch(color='red', label='Departure & Blocked Arrival (u=1, v=0)'),
                            mpatches.Patch(color='green', label='Departure (u=1)'),
                            mpatches.Patch(color='purple', label='Arrival Blocking (v=0)')]
    ax.legend(handles=color_legend_handles, loc='lower right', fontsize=10)

    # Set labels and title
    ax.set_xlabel('Time', fontsize=12)
    ax.set_ylabel('Queue Length', fontsize=12)
    ax.set_title(f'Queue Length Over Time, Initial length i = {i_init}', fontsize=14)
    ax.grid(True)

    # Show plot
    plt.savefig(f'Experiment{experiment}_queue{i_init}_learning.png',dpi=300)
    plt.show()
